In [1]:
from bs4 import BeautifulSoup, Comment
import pandas as pd
import urllib

In [2]:
%%time
def get_teams(soup):
    '''Takes BeautifulSoup object of NBA teams page and returns
       list of links to team pages.'''
   
    for link in soup.find_all('a'):
        team = link.text
        if 'teams' in link.get('href'):
            yield link.get('href')
        if team == 'Washington Wizards':
            break

site = 'https://www.basketball-reference.com/teams/'
page= urllib.request.urlopen(site)
soup = BeautifulSoup(page, 'lxml')

teamlinks = list(get_teams(soup))[1:]

Wall time: 1.8 s


In [3]:
%%time
def get_players(teams, year):
    '''From list of teams, get list of all players on that 
       team for a given year. This is only set up to work
       on seasons from 2009-10 on. There may be errors going
       further back.'''
    
    year = str(year)
    
    for team in teams:
        link = 'https://www.basketball-reference.com' + team + year + '.html'
        if 'NJN' in link:
            if year > '2012':
                link = 'https://www.basketball-reference.com/teams/BRK/' + year + '.html'
            else:
                link = 'https://www.basketball-reference.com/teams/NJN/' + year + '.html'
        if 'CHA' in link:
                if year > '2014':
                    link = 'https://www.basketball-reference.com/teams/CHO/' + year + '.html'
                elif year < '2003':
                    link = 'https://www.basketball-reference.com/teams/CHH/' + year + '.html'
                else:
                    'https://www.basketball-reference.com/teams/CHA/'+ year + '.html'
                
        if 'NOH' in link:
            if year > '2013':
                link = 'https://www.basketball-reference.com/teams/NOP/' + year + '.html'
            else:
                 link = 'https://www.basketball-reference.com/teams/NOH/' + year + '.html'
               
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'lxml')

        for i in soup.find_all('table')[0].find_all('a'):
            if 'players' in i['href']:
                yield (i.text, i['href'])

Wall time: 0 ns


In [4]:
%%time
player_set = set()
for year in range(2010,2020):
    new_set = set(get_players(teamlinks,year))
    if len(player_set) == 0:
        player_set = new_set
    else:
        player_set.update(new_set)
    print(year, len(player_set))

2010 442
2011 523
2012 611
2013 692
2014 770
2015 852
2016 926
2017 1014
2018 1133
2019 1238
Wall time: 10min 15s


In [5]:
player_list = list(player_set)

In [7]:
x = 1
all_df = pd.DataFrame()
for player in player_list:
    reference_site = 'https://www.basketball-reference.com' + player[1]
    page = urllib.request.urlopen(reference_site)
    soup = BeautifulSoup(page, 'lxml')
    per_game_table = soup.table
    
    # BBR comments out a lot of the tables, and this function
    # pulls information from those tables
    # Taken from:
    # https://stackoverflow.com/questions/49766150/
    # when-scraping-data-from-basketball-reference-
    # how-come-certain-tables-are-comment
    all_list = []
    for comment in soup.find_all(string=lambda text:isinstance(text,Comment)):
        data=BeautifulSoup(comment, "lxml")
        for items in data.select("table.row_summable tr"):
            tds = [item.get_text(strip=True) for item in items.select("th,td")]
            all_list.append(tds)
    # get index of table headers
    df_starts = []
    for i, row in enumerate(all_list):
        if row[0] == 'Season':
            df_starts.append(i)

    base = list(player)
    # create table of advanced stats
    table = [row for row in all_list[df_starts[3]:df_starts[4]]]

    # convert each table to a DataFrame and add to list of dataframes
    cols = ['Player', 'href'] + table[0]
    player_df = pd.DataFrame(table[1:], columns=table[0])
    player_df['Player'] = player[0]
    player_df['href'] = player[1]
    
    # drop rows with totals and incorrect values
    
    
    all_df = all_df.append(player_df)
    print(x, player[0])
    x += 1


1 John Holland
2 Quinn Cook
3 Andre Dawkins
4 Pierre Jackson
5 Richard Jefferson
6 Eric Moreland
7 Christian Eyenga
8 Gerald Green
9 Terrence Williams
10 Kevin Martin
11 Scotty Hopson
12 Cleanthony Early
13 Shawn Long
14 Chimezie Metu
15 Maxi Kleber
16 Greg Monroe
17 Henry Walker
18 Allen Crabbe
19 Tony Snell
20 Melvin Frazier
21 Patrick Patterson
22 Jeff Adrien
23 Donte DiVincenzo
24 Jaylen Brown
25 Rakeem Christmas
26 Josh Harrellson
27 James Posey
28 Steve Nash
29 Luke Harangody
30 D'Angelo Russell
31 Patrick McCaw
32 Noah Vonleh
33 Jordan McRae
34 Peyton Siva
35 Darryl Watkins
36 Delonte West
37 Al Harrington
38 Rodrigue Beaubois
39 Lonnie Walker
40 Isaiah Canaan
41 Lazar Hayward
42 Kobe Bryant
43 Andrew Bogut
44 Quincy Pondexter
45 JaKarr Sampson
46 Nathan Jawai
47 Jeremy Tyler
48 Tim Thomas
49 Tim Quarterman
50 Jacob Wiley
51 Emanuel Terry
52 Michael Kidd-Gilchrist
53 Steve Novak
54 Chris Wright
55 Solomon Hill
56 Isaac Humphries
57 Roy Hibbert
58 MarShon Brooks
59 Mario Chalmers

465 Erick Green
466 Nemanja Bjelica
467 Eric Gordon
468 Mitch McGary
469 Jason Smith
470 Michael Gbinije
471 Sasha Kaun
472 Ray Spalding
473 Kosta Koufos
474 Matthew Dellavedova
475 DeAndre Jordan
476 Ricky Ledo
477 Delon Wright
478 Alize Johnson
479 Zydrunas Ilgauskas
480 Daniel Ochefu
481 James Nunnally
482 Shake Milton
483 Chris Quinn
484 Francisco Elson
485 Lauri Markkanen
486 Jawad Williams
487 Archie Goodwin
488 Quincy Acy
489 Nenad Krstić
490 Craig Smith
491 Kendrick Perkins
492 Ben Gordon
493 Hamidou Diallo
494 Ryan Reid
495 Malcolm Lee
496 Jake Layman
497 Marcus Cousin
498 Charles Cooke
499 Thomas Welsh
500 Solomon Jones
501 Devean George
502 Andre Iguodala
503 Orlando Johnson
504 Steven Hunter
505 Dejounte Murray
506 C.J. Williams
507 Eugene Jeter
508 Hakim Warrick
509 Diamond Stone
510 Cole Aldrich
511 Devonte' Graham
512 Terrence Ross
513 Lester Hudson
514 Jonny Flynn
515 Hollis Thompson
516 Mikki Moore
517 Vítor Faverani
518 Christian Wood
519 Dāvis Bertāns
520 Jeremy Lin


926 Jevon Carter
927 Eric Dawson
928 Tahjere McCall
929 Justin Anderson
930 Sam Dekker
931 Oleksiy Pecherov
932 Kristaps Porziņģis
933 Bradley Beal
934 Larry Sanders
935 Julius Randle
936 Renaldo Balkman
937 Wendell Carter
938 Dan Gadzuric
939 Brandon Sampson
940 Ben Moore
941 Dillon Brooks
942 Manny Harris
943 Willie Warren
944 Taurean Waller-Prince
945 Timofey Mozgov
946 Wayne Ellington
947 Zach Randolph
948 Miroslav Raduljica
949 Zach Collins
950 Derrick Brown
951 Joe Johnson
952 DaJuan Summers
953 Maalik Wayns
954 Trae Young
955 Nick Young
956 J.R. Giddens
957 Tony Allen
958 Nick Calathes
959 Shabazz Napier
960 Trey Johnson
961 Charlie Villanueva
962 Sviatoslav Mykhailiuk
963 Omari Spellman
964 PJ Dozier
965 Marcus Derrickson
966 Kyle Lowry
967 Morris Peterson
968 Troy Williams
969 Monta Ellis
970 Pops Mensah-Bonsu
971 Michael Redd
972 Kwame Brown
973 Brad Miller
974 Ron Baker
975 Miloš Teodosić
976 Joe Chealey
977 Brandon Goodwin
978 Joe Smith
979 Corey Brewer
980 Markieff Morris


In [8]:
# filter out all columns that are not single season like 
# career or team sums
regex = '[0-9]{4}-[0-9]{2}'
filt = all_df['Season'].str.contains(regex)

final_df = all_df[filt]

In [17]:
final_df.reset_index(inplace=True, drop=True)

In [10]:
final_df.to_csv('advanced.csv')